# Reshape by ACS table

In [1]:
import numpy as np
import pandas as pd
import intake
import os

In [51]:
edu = pd.read_csv('../data/Census/educational_attainment_tract.csv')
edu2 = pd.read_csv('../data/Census/educational_attainment_tract_2015_2017.csv')

In [50]:
edu.year.value_counts()

2017    46920
2016    46920
2015    46920
2014    46920
2013    46920
2012    46920
2011    46920
2010    46920
Name: year, dtype: int64

In [52]:
edu2.year.value_counts()

2017    149718
2016    149718
2015    149718
Name: year, dtype: int64

In [54]:
len(edu2)

449154

In [53]:
len(edu2.drop_duplicates())

449154

In [57]:
edu2[(edu2.GEOID=='06037141400')]

,GEOID,NAME,variable,estimate,moe,year


In [2]:
df = pd.read_parquet('../data/raw_census_long.parquet')
df = df[df.GEOID=='06037141400']

In [40]:
orig =pd.read_parquet('../data/raw_census_long.parquet')
orig = orig[orig.table=='edu']

In [48]:
orig[(orig.second_var=='pct_hsplus') & (orig.year==2017)].variable.value_counts()

S1501_C01_014E    2346
Name: variable, dtype: int64

In [3]:
df.table.value_counts()

emp             640
income          160
edu             160
food             48
povfam           16
pubassist        16
housing           8
pop               8
aggpubassist      8
Name: table, dtype: int64

## Employment

In [4]:
emp = df[df.table=='emp']

In [6]:
def emp_type(row):
    if (row.main_var=='pop'):
        return 'number'
    elif (row.main_var != 'pop'):
        return 'percent'
    
emp['var_type'] = emp.apply(emp_type, axis = 1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [17]:
emp_pop = emp[emp.main_var=='pop']

emp_dfs = {}

# For each table, do a merge with pop numbers, convert the percents back to numbers and save into dictionary
for subset in ['lf', 'epr', 'unemp']:
    new_pct_col = "pct"
    new_num_col = "num"
    subset_df = emp[emp.main_var == subset]
    merged = pd.merge(emp_pop16, subset_df, on = ['GEOID', 'year', 'table', 'second_var'])
    merged = merged.drop(columns = ['variable_x', 'main_var_x', 'new_var_x'])
    merged.rename(columns = {'value_x': 'num_pop', 'value_y': new_pct_col}, inplace = True)
    merged[new_pct_col] = merged[new_pct_col] / 100
    merged[new_num_col] = merged.num_pop * merged[new_pct_col]
    merged = merged[['GEOID', 'year', 'table', 'second_var', 'variable_y', 'main_var_y', 'new_var_y', new_pct_col, new_num_col]]
    merged.rename(columns = {'variable_y': 'variable', 'main_var_y': 'main_var', 'new_var_y': 'new_var'}, inplace = True)
    emp_dfs[subset] = merged

In [18]:
emp_pop['num'] = emp_pop.value
emp_pop['pct'] = 1
emp_pop = emp_pop.drop(columns = ['value', 'var_type'])
emp_pop.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,GEOID,year,variable,table,main_var,second_var,new_var,num,pct
26880,06037141400,2010,S2301_C01_001E,emp,pop,total_pop16,pop_total_pop16,3697.0,1
26881,06037141400,2010,S2301_C01_010E,emp,pop,white,pop_white,2904.0,1
26882,06037141400,2010,S2301_C01_011E,emp,pop,black,pop_black,156.0,1
26883,06037141400,2010,S2301_C01_012E,emp,pop,amerind,pop_amerind,0.0,1
26884,06037141400,2010,S2301_C01_013E,emp,pop,asian,pop_asian,372.0,1


In [24]:
# Append the emp dfs together to be long
appended = emp_pop

for key, value in emp_dfs.items():
    appended = appended.append(value, sort = False)

## Income

In [27]:
income = df[df.table=='income']

In [28]:
def income_type(row):
    if (row.main_var=='hh') & (row.second_var=='hh'):
        return 'number'
    elif (row.main_var == 'medincome'):
        return 'dollar'
    elif (row.main_var=='hh') & (row.second_var != 'hh') & (row.year <= 2016):
        return 'percent'
    elif (row.main_var=='hh') & (row.second_var != 'hh') & (row.year >= 2017):
        return 'number'
    
income['var_type'] = income.apply(income_type, axis = 1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [31]:
# Create a denominator column. Use this to convert percent values into numbers.
income['denom'] = income.apply(lambda row: row.value if row.new_var=='hh_total' else np.nan, axis = 1)
income['denom'] = income['denom'].fillna(income.groupby(['GEOID', 'year'])['denom'].transform('max'))
income.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,GEOID,year,variable,value,table,main_var,second_var,new_var,var_type,denom
6720,06037141400,2010,S1903_C01_001E,1925.0,income,hh,total,hh_total,percent,1925.0
6721,06037141400,2010,S1903_C01_002E,81.7,income,hh,white,hh_white,percent,1925.0
6722,06037141400,2010,S1903_C01_003E,2.2,income,hh,black,hh_black,percent,1925.0
6723,06037141400,2010,S1903_C01_004E,0.0,income,hh,amerind,hh_amerind,percent,1925.0
6724,06037141400,2010,S1903_C01_005E,8.9,income,hh,asian,hh_asian,percent,1925.0


In [32]:
# Replace values that were percents with numbers
income['pct'] = income.apply(lambda row: (row.value / 100) if row.var_type=='percent' 
                             else (row.value / row.denom), axis = 1)

income['num'] = income.apply(lambda row: row.value if row.var_type=='number' else (row.pct * row.denom), axis = 1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [34]:
drop_me = ['value', 'var_type', 'denom']
income = income.drop(columns = drop_me)

,GEOID,year,variable,table,main_var,second_var,new_var,pct,num
6720,06037141400,2010,S1903_C01_001E,income,hh,total,hh_total,19.250000,37056.250
6721,06037141400,2010,S1903_C01_002E,income,hh,white,hh_white,0.817000,1572.725
6722,06037141400,2010,S1903_C01_003E,income,hh,black,hh_black,0.022000,42.350
6723,06037141400,2010,S1903_C01_004E,income,hh,amerind,hh_amerind,0.000000,0.000
6724,06037141400,2010,S1903_C01_005E,income,hh,asian,hh_asian,0.089000,171.325
6725,06037141400,2010,S1903_C01_006E,income,hh,pacis,hh_pacis,0.000000,0.000
6726,06037141400,2010,S1903_C01_007E,income,hh,other,hh_other,0.042000,80.850
6727,06037141400,2010,S1903_C01_008E,income,hh,race2,hh_race2,0.030000,57.750
6728,06037141400,2010,S1903_C01_009E,income,hh,hisp,hh_hisp,0.100000,192.500
6729,06037141400,2010,S1903_C01_010E,income,hh,nonhisp,hh_nonhisp,0.773000,1488.025


## Education

In [35]:
edu = df[df.table=='edu']

In [36]:
def edu_type(row):
    if (row.second_var.find('pct') != -1) or (row.second_var.find('pov') != -1):
        return 'percent'
    elif row.second_var.find('medearning') != -1:
        return 'dollar'
    elif (row.second_var.find('pct') == -1) & (row.second_var.find('medearning') == -1):
        return 'number'
    
edu['var_type'] = edu.apply(edu_type, axis = 1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
# Create a denominator column. Use this to convert percent values into numbers.
edu['denom'] = edu.apply(lambda row: row.value if row.new_var=='hh_total' else np.nan, axis = 1)
edu['denom'] = edu['denom'].fillna(edu.groupby(['GEOID', 'year'])['denom'].transform('max'))
edu.head()

In [38]:
# Replace values that were percents with numbers
income['pct'] = income.apply(lambda row: (row.value / 100) if row.var_type=='percent' 
                             else (row.value / row.denom), axis = 1)

income['num'] = income.apply(lambda row: row.value if row.var_type=='number' else (row.pct * row.denom), axis = 1)

,GEOID,year,variable,value,table,main_var,second_var,new_var,var_type
6720,06037141400,2010,S1501_C01_006E,3428.0,edu,pop,total_pop25,pop_total_pop25,number
6721,06037141400,2010,S1501_C01_007E,2.1,edu,pop,hs9,pop_hs9,number
6722,06037141400,2010,S1501_C01_008E,3.1,edu,pop,hs12,pop_hs12,number
6723,06037141400,2010,S1501_C01_009E,12.5,edu,pop,hs,pop_hs,number
6724,06037141400,2010,S1501_C01_010E,17.4,edu,pop,college,pop_college,number
6725,06037141400,2010,S1501_C01_011E,4.3,edu,pop,aa,pop_aa,number
6726,06037141400,2010,S1501_C01_012E,33.5,edu,pop,ba,pop_ba,number
6727,06037141400,2010,S1501_C01_013E,27.0,edu,pop,ma,pop_ma,number
6728,06037141400,2010,S1501_C01_014E,94.7,edu,pop,pct_hsplus,pop_pct_hsplus,percent
6729,06037141400,2010,S1501_C01_015E,60.6,edu,pop,pct_baplus,pop_pct_baplus,percent


In [39]:
edu[edu.year==2017]

,GEOID,year,variable,value,table,main_var,second_var,new_var,var_type
329330,06037141400,2017,S1501_C01_006E,3800.0,edu,pop,total_pop25,pop_total_pop25,number
331676,06037141400,2017,S1501_C01_007E,30.0,edu,pop,hs9,pop_hs9,number
334022,06037141400,2017,S1501_C01_008E,73.0,edu,pop,hs12,pop_hs12,number
336368,06037141400,2017,S1501_C01_009E,448.0,edu,pop,hs,pop_hs,number
338714,06037141400,2017,S1501_C01_010E,603.0,edu,pop,college,pop_college,number
341060,06037141400,2017,S1501_C01_011E,324.0,edu,pop,aa,pop_aa,number
343406,06037141400,2017,S1501_C01_012E,1180.0,edu,pop,ba,pop_ba,number
345752,06037141400,2017,S1501_C01_013E,1142.0,edu,pop,ma,pop_ma,number
348098,06037141400,2017,S1501_C01_014E,NaN,edu,pop,pct_hsplus,pop_pct_hsplus,percent
350444,06037141400,2017,S1501_C01_015E,NaN,edu,pop,pct_baplus,pop_pct_baplus,percent


In [ ]:
# Replace values that were percents with numbers
def edu_adjust(row):
    if (row.var_type=='percent'):
        return row.value / 100
    else:
        return row.value

edu['value_adj'] = edu.apply(edu_adjust, axis = 1)

In [ ]:
keep_me = ['GEOID', 'year', 'variable', 'table', 'main_var', 'second_var', 
           'new_var', 'value_adj', 'var_type']

edu = edu[keep_me]

## Population and Housing Units

In [ ]:
pop_housing = df[(df.table=='pop') | (df.table=='housing') ]

In [ ]:
def pop_housing_type(row):
    if row.second_var.find('tot') != -1:
        return 'number'
    
pop_housing['var_type'] = pop_housing.apply(pop_housing_type, axis = 1)

In [ ]:
# Create value_adj column (no adjustments needed, but need this so later can incorporate into loop)
def pop_housing_adjust(row):
    if row.var_type=='percent':
        return row.value / 100
    else:
        return row.value

pop_housing['value_adj'] = pop_housing.apply(pop_housing_adjust, axis = 1)

In [ ]:
keep_me = ['GEOID', 'year', 'variable', 'table', 'main_var', 'second_var', 
           'new_var', 'value_adj', 'var_type']

pop_housing = pop_housing[keep_me]

## Poverty

In [ ]:
povfam = df[df.table=='povfam']

In [ ]:
def povfam_type(row):
    if row.main_var.find('pov') != -1:
        return 'percent'
    else:
        return 'number'
    
povfam['var_type'] = povfam.apply(povfam_type, axis = 1)

In [ ]:
povfam['denom'] = povfam.apply(lambda row: row.value if row.new_var=='families' else np.nan, axis = 1)
povfam['denom'] = povfam['denom'].fillna(povfam.groupby(['GEOID', 'year'])['denom'].transform('max'))

In [ ]:
# Replace values that were percents with numbers
def povfam_adjust(row):
    if (row.var_type=='percent'):
        return (row.value/100) * row.denom
    else:
        return row.value

povfam['value_adj'] = povfam.apply(povfam_adjust, axis = 1)

In [ ]:
# Structure df so that there are number and percent columns, with values filled in
povfam['value_pct'] = povfam.value_adj / povfam.denom

In [ ]:
povfam

In [ ]:
"""
Food stamps table has inconsistent values.
2010-2014, hh in poverty is given in %. same as 
2015-2017, hh in poverty is given as numbers.
Do each table separately.

Store column that tells me if they are numbers, percents, or dollars.
"""
food = df[df.table=='food']
food[food.year==2012]

In [ ]:
df.to_stata('../data/raw_census_long.dta')

In [ ]:
pubassist = df[df.table=='pubassist']
# These are number of families that received food stamps
# Need to derive percents

In [ ]:
pubassist